<a href="https://colab.research.google.com/github/drppdrpp/drppdrpp.github.io/blob/master/%E5%9B%9B%E6%9C%AC%E5%80%A4%E6%A4%9C%E7%B4%A2(%E3%82%B9%E3%83%97%E3%82%B7%E9%80%A3%E6%90%BA)Yfinance_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
!pip install --upgrade -q gspread
import gspread

#@title Google Drive接続と認証
# 参考サイトhttps://www.teijitaisya.com/python-gspread/
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

#@title スプレッドシートを開く、特定
# # スプレッドシートを開く 名前 or URL可
filename = '四本値検索(Colab連携)'
ss = gc.open(filename)

# シートを特定する（シート名で特定）
st = ss.worksheet("読込用")

# import pandas as pd

# https://github.com/pkout/yahoo_finance_api2
!pip install yahoo_finance_api2
import sys
import datetime
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
# 日本の営業日
!pip install jpbizday
import jpbizday

import datetime
from datetime import datetime as dt

from logging import exception
from time import sleep
import requests
import lxml.html
import re

# 開始日(必須)	終わり日(有れば)	コード(必須)

     |████████████████████████████████| 40 kB 2.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [253]:
# 今年の東証休業日取得(祝日と年末年始の休業日)
def get_tosho_holidays():
  tosho_holidays_list = []
  # スクレイピング版は、今年分しかとれないので、ライブラリを使いつつ東証の独自の休業日は手入力で追加
  tosho_holidays_list.extend(["2015-01-01","2015-01-02","2015-01-03","2015-12-31",\
                              "2016-01-01","2016-01-02","2016-01-03","2016-12-31",\
                              "2017-01-01","2017-01-02","2017-01-03","2017-12-31",\
                              "2018-01-01","2018-01-02","2018-01-03","2018-12-31",\
                              "2019-01-01","2019-01-02","2019-01-03","2019-12-31",\
                              "2020-01-01","2020-01-02","2020-01-03","2020-12-31",\
                              "2021-01-01","2021-01-02","2021-01-03","2021-12-31",\
                              "2022-01-01","2022-01-02","2022-01-03","2022-12-31"])
  return tosho_holidays_list

# 土日と東証休日を判定
def today_is_bizday(tosho_holidays_list, today_datetime):

  SATURDAY = 5
  SUNDAY = 6
  today_is_bizday = True
  try:
    # 土曜or日曜は実行しない(月曜が0~日曜が6)
    day_of_week = today_datetime.date().weekday()
    if (day_of_week == SATURDAY) or (day_of_week == SUNDAY):
      today_is_bizday = False

    today = str(today_datetime.date())
    for i in tosho_holidays_list:
      if today == i:
        today_is_bizday = False
    return today_is_bizday

  except Exception as e:
      print(e)

# 営業日ではない場合に+1日送り続ける
def returnBizDay(biz_date):
  tosho_holidays_list = get_tosho_holidays()
  while jpbizday.is_bizday(biz_date) == False or today_is_bizday(tosho_holidays_list, biz_date) == False:
      biz_date = biz_date + datetime.timedelta(days=1)
  return biz_date

# GUGDの初日分は、前日の終値が必要なため、前日を別で取得
def returnBackBizDay(biz_date):
  tosho_holidays_list = get_tosho_holidays()
  biz_date = biz_date + datetime.timedelta(days=-1)
  while jpbizday.is_bizday(biz_date) == False or today_is_bizday(tosho_holidays_list, biz_date) == False:
      biz_date = biz_date + datetime.timedelta(days=-1)
  return biz_date


def convert_1d_to_2d(list_1d, COLUMNS):
  tmp =[]
  for i in range(0, len(list_1d), COLUMNS):
    tmp.append(list_1d[i:i + COLUMNS])
  return tmp

In [279]:
success_counts = 0
failure_counts = 0
datecode_lists = st.get_all_values()

for i in range(len(datecode_lists)):
  if i == 0:
    continue

  input_start_var = datecode_lists[i][0]

  if datecode_lists[i][1] == '':
    input_end_var = input_start_var
  else:  
    input_end_var = datecode_lists[i][1]

  input_code = datecode_lists[i][2]
  stock_name = datecode_lists[i][3]
  factor = datecode_lists[i][4]
  # 問題はザラ場時間に動かすと、連動したリアルタイムで取得され、さらに、このapiでは時間調整ができない

  year = re.search(r"(\d+)",input_start_var) 
  monthdate = re.search(r"([\d]+)[^\d]*([\d]+)[^\d]*$",input_start_var) 
  year = int(year.group(1))
  month = int(monthdate.group(1))
  day = int(monthdate.group(2))

  start_day = datetime.datetime(year=year, month=month, day=day)
  # star_day = datetime.datetime.now() + datetime.timedelta(hours=0)
  datetimenow = datetime.datetime.now() + datetime.timedelta(hours=+9) - start_day# なぜか一日追加される
  # print(f"今から {datetimenow} 戻ると、指定時間 {start_day}")
  days = datetimenow.days

  #GUGD用 前日が休業日なら更に数日戻して営業日を特定する
  forward_start = returnBackBizDay(start_day)
  datetimenow2 = datetime.datetime.now() + datetime.timedelta(hours=+9) - forward_start
  # print(f"今から {datetimenow2} 戻ると、前日前の営業日まで戻して {forward_start}")
  days2 = datetimenow2.days

  year = re.search(r"(\d+)", input_end_var) 
  month_day = re.search(r"([\d]+)[^\d]*([\d]+)[^\d]*$", input_end_var) 
  year = int(year.group(1))
  month = int(month_day.group(1))
  day = int(month_day.group(2))

  end_day = datetime.datetime(year=year, month=month, day=day)
  # dt2 = datetime.datetime.now() + datetime.timedelta(hours=0)
  end_in_biz = returnBizDay(end_day)

  datetime_end = datetime.datetime.now() + datetime.timedelta(hours=+9) - end_in_biz# なぜか一日追加される
  # print(f"今から {datetime_end} 戻ると、指定時間 {end_in_biz}")

  end_in_biz = end_in_biz.strftime('%Y-%m-%d')
  try:
    my_share = share.Share(f'{input_code}.T')
    symbol_data = None
    try:
      symbol_data = my_share.get_historical(
        share.PERIOD_TYPE_DAY, days,
        share.FREQUENCY_TYPE_DAY, 1) 
    except YahooFinanceError as e:
      print(e.message)
      # sys.exit(1)

    # timesmtampを消して(str)dateとして追加
    ts = symbol_data["timestamp"]
    converted_list = []
    for i in ts:
      day = datetime.datetime.fromtimestamp(i/1000 + 32400)#32400秒=9時間
      converted_list.append(day.strftime('%Y-%m-%d'))
    symbol_data.pop("timestamp")
    symbol_data["date"] = converted_list
    date_list = symbol_data["date"]# 辞書内は、追加順になってしまう
    # end営業日が何個目にあるのか？
    count_num = 0
    for i in date_list:
      if  end_in_biz == i:
        break
      count_num += 1


    # 日経用
    my_share_nikkei = share.Share(f'^N225')
    nikkei_data = None
    try:
      nikkei_data = my_share_nikkei.get_historical(
        share.PERIOD_TYPE_DAY, days,
        share.FREQUENCY_TYPE_DAY, 1) 
    except YahooFinanceError as e:
      print(e.message)
      # sys.exit(1)


    # GUGD用前日
    my_share_for_gugd = share.Share(f'{input_code}.T')
    symbol_data_for_gugd = None
    try:
      symbol_data_for_gugd = my_share_for_gugd.get_historical(
        share.PERIOD_TYPE_DAY, days2,
        share.FREQUENCY_TYPE_DAY, 1) 
    except YahooFinanceError as e:
      print(e.message)
      # sys.exit(1)

    data = []
    days_list=[]
    i = 0
    count_num +=1
    
    for i in range(count_num):
    
      data.append(symbol_data["date"][i])
      data.append(input_code)
      data.append(stock_name)
      data.append(factor)
      open = int(symbol_data["open"][i])
      close = int(symbol_data["close"][i])
      latest_close = int(symbol_data["close"][count_num-1])#その日からend_dateまでの%

      data.append(open)
      data.append(int(symbol_data["high"][i]))
      data.append(int(symbol_data["low"][i]))
      data.append(close)
      data.append(symbol_data["volume"][i])
      data.append(round((close - open) / open * 100, 2))
      data.append(round((latest_close - open) / open * 100, 2))
      open_nikkei = int(nikkei_data["open"][i])
      close_nikkei = int(nikkei_data["close"][i])
      data.append(round((close_nikkei - open_nikkei) / open_nikkei * 100, 2))
      close_gugd = int(symbol_data_for_gugd["close"][i])
      open_gugd = int(symbol_data_for_gugd["open"][i+1])
      data.append(round((open_gugd - close_gugd) / close_gugd * 100, 2))

      data.append(i + 1)
      days_list.append(data)

      COLUMNS = 14
      result = convert_1d_to_2d(data, COLUMNS)
      
    print(result)
    ss.values_append('結果用', {"valueInputOption": "USER_ENTERED"}, {"values": result})
    sleep(1)
    success_counts += 1
      
  except Exception as e:

    print(f'取得不可エラーは {i+1}行目 → {input_code}')
    result = [[input_start_var, input_code, stock_name, factor, "取得不可エラー"]]
    ss.values_append('結果用', {"valueInputOption": "USER_ENTERED"}, {"values": result})
    failure_counts +=1
  if failure_counts == 100:
    break

print('----------------------------------')
print(f'〇出力成功が {success_counts}個です')
print(f'×出力失敗が {failure_counts}個です、計100個に達すると強制終了します')
# ['DATE','CODE','銘柄名','材料内容','Open','High','Low','Close','Volume',    '始終(%)','初始値→毎終値(%)','(日経)初始値→毎終値(%)','GU_GD(%)','何日目']]

[['2022-11-07', '6054', 'リブセンス', 'テスト', 282, 282, 272, 278, 111100, -1.42, -1.42, 0.44, 1.44, 1]]
[['2022-11-02', '7777', 'mato', 'tst', 300, 301, 299, 300, 292000, 0.0, 0.0, 0.37, -0.66, 1]]
[['2022-11-07', '6054', 'リブセンス', 'テスト', 282, 282, 272, 278, 111100, -1.42, 0.35, 0.44, 1.44, 1], ['2022-11-08', '6054', 'リブセンス', 'テスト', 277, 289, 277, 284, 218900, 2.53, 2.17, 0.56, -0.36, 2], ['2022-11-09', '6054', 'リブセンス', 'テスト', 285, 287, 277, 279, 136700, -2.11, -0.7, -0.6, 0.35, 3], ['2022-11-10', '6054', 'リブセンス', 'テスト', 275, 285, 275, 283, 190000, 2.91, 2.91, -0.05, -1.43, 4]]
Not Found: No data found, symbol may be delisted
取得不可エラーは 5行目 → 123
----------------------------------
〇出力成功が 3個です
×出力失敗が 1個です、計100個に達すると強制終了します


In [ ]:
# print(type(symbol_data))

# どうやらこの単位をサポート。固定されている。
# PERIOD_TYPE_DAY = 'day'
# PERIOD_TYPE_WEEK = 'week'
# PERIOD_TYPE_MONTH = 'month'
# PERIOD_TYPE_YEAR = 'year'

# Valid frequencies: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
# FREQUENCY_TYPE_MINUTE = 'm'
# FREQUENCY_TYPE_HOUR = 'h'
# FREQUENCY_TYPE_DAY = 'd'
# FREQUENCY_TYPE_WEEK = 'wk'
# FREQUENCY_TYPE_MONTH = 'mo'

